In [3]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd


# Recuperiamo l'elenco aggiornato di dataflow ISTAT in formato XML
url_dataflow = "https://esploradati.istat.it/SDMXWS/rest/dataflow"
response = requests.get(url_dataflow)
response.raise_for_status()

# Parsing del contenuto XML
root = ET.fromstring(response.content)

# Namespace con 'xml' incluso per accedere ai titoli in italiano
ns = {
    "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
    "xml": "http://www.w3.org/XML/1998/namespace"
}

# Estrazione dei dataset
dataflows = []
for df in root.findall(".//structure:Dataflow", ns):
    dataset_id = df.attrib['id']
    name_it = df.find(".//common:Name[@xml:lang='it']", ns)
    name_en = df.find(".//common:Name[@xml:lang='en']", ns)
    name = name_it.text if name_it is not None else (name_en.text if name_en is not None else "")
    dataflows.append({"dataset_id": dataset_id, "title": name})

df = pd.DataFrame(dataflows)

# Filtro per dataset rilevanti a tema economico/sociale
parole_chiave = "coltiv|prodott|produzione|superfici|dop|igp|qualità|famigl|occup|lav|imprese|valore aggiunto|reddito|consum|pil"
df_filtrato = df[df["title"].str.lower().str.contains(parole_chiave, na=False)].reset_index(drop=True)

In [4]:
df_filtrato

,dataset_id,title
0,101_1015,Coltivazioni
1,101_1015_DF_DCSP_COLTIVAZIONI_1,Superfici e produzione - dati in complesso
2,101_1015_DF_DCSP_COLTIVAZIONI_2,Superfici e produzione - dati in complesso - prov
3,101_1030,Prodotti agroalimentari di qualità DOP/IGP e STG
4,101_1030_DF_DCSP_DOPIGP_2,Operatori per prodotto
...,...,...
822,EMP,Occupazione
823,IND,Indice della produzione industriale
824,PPI,Prezzi alla produzione
825,SDDS_PLUS_CPI_DF_167_744_1,Prezzi al consumo


## ottenere l’elenco delle strutture dati SDMX da ISTAT (/datastructure)

In [ ]:
# ottenere l’elenco delle strutture dati SDMX da ISTAT (/datastructure)
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Scarica l'elenco di tutte le datastructure disponibili da ISTAT SDMX
url = "https://esploradati.istat.it/SDMXWS/rest/datastructure"
response = requests.get(url)
root = ET.fromstring(response.content)

# Namespace XML usati da SDMX
ns = {
    "message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common"
}

# Estrae tutte le strutture (id, agencyID, version, nome IT/EN)
records = []
for dsd in root.findall(".//structure:DataStructure", ns):
    ref_id = dsd.attrib.get("id")
    agency_id = dsd.attrib.get("agencyID")
    version = dsd.attrib.get("version")
    name_it = name_en = ""
    for name in dsd.findall("common:Name", ns):
        if name.attrib.get("{http://www.w3.org/XML/1998/namespace}lang") == "it":
            name_it = name.text
        elif name.attrib.get("{http://www.w3.org/XML/1998/namespace}lang") == "en":
            name_en = name.text
    records.append({
        "id": ref_id,
        "agencyID": agency_id,
        "version": version,
        "name_it": name_it,
        "name_en": name_en
    })


In [11]:
# Crea il DataFrame
df_structures = pd.DataFrame(records)

# Visualizza le prime righe
(df_structures.head(10))

,id,agencyID,version,name_it,name_en
0,ECOFIN_DSD,IMF,1.0,,ECOFIN Data Structure Definition
1,BES_TERRIT,IT1,1.1,SIR indicators territorial level,Indicatori SIR a livello territoriale
2,COMP_FRA_IND_MUNICIPAL,IT1,1.0,Indici di fragilità,Fragility indexes
3,DCAR_ATT_NOTAR,IT1,1.0,Atti notarili,Notarial deeds
4,DCAR_CONTECON_ASLAO,IT1,1.0,"Conto economico delle Asl, aziende ospedaliere...","Local health units, hospital companies, univer..."
5,DCAR_CONTECON_CC,IT1,1.0,Conto economico delle camere di commercio - euro,Chambers of commerce profit and loss account -...
6,DCAR_CONTECON_UNI,IT1,1.0,"Conto economico delle università, politecnici ...","Universities, polytechnic institutes and insti..."
7,DCAR_CONV_ATT_NOT,IT1,1.0,Convenzioni contenute negli atti notarili,Agreements contained in notarial deeds
8,DCAR_DEC_AMM,IT1,1.0,Decisioni di giustizia amministrativa,Administrative justice decisions
9,DCAR_DIPENTIPREV,IT1,1.0,Numero dei dipendenti degli enti previdenziali...,Number of employees of social security funds 3pop


In [12]:
# (Facoltativo) Filtro per strutture che riguardano temi economici/sociali
parole_chiave = "occup|impres|pil|valore aggiunto|reddito|famigl|consum|spesa|produzion|agric|forest|lavor|salari|popolaz|coesi|svilupp|ambiente"
df_filtrato = df_structures[
    df_structures['name_it'].str.lower().str.contains(parole_chiave, na=False) |
    df_structures['name_en'].str.lower().str.contains(parole_chiave, na=False)
].reset_index(drop=True)

# Stampa le strutture filtrate
(df_filtrato[['id', 'name_it', 'name_en']])

,id,name_it,name_en
0,DCAR_PROC_CIV,"Procedimenti civili - lavoro, previdenza e ass...","Civil proceedings - labour, social security an..."
1,DCAT_CENSAGRIC2020,Censimento agricoltura 2020,Agriculture census 2020
2,DCAT_CENSAGRIC2020_01,Censimento agricoltura 2020,Agriculture census 2020
3,DCAT_CENSAGRIC2020_AGE,Censimento agricoltura 2020,Agriculture census 2020
4,DCAT_CENSAGRIC2020_AGE_ASS,Censimento agricoltura 2020,Agriculture census 2020
...,...,...,...
312,PERCEPIT213747,Famiglie per capacità di arrivare a fine mese,Households by ability to make ends meet
313,PERCEPIT263604,Famiglie per capacità di arrivare a fine mese,Households by ability to make ends meet
314,PERCEPIT406502,Famiglie per capacità di arrivare a fine mese,Households by ability to make ends meet
315,PERCEPIT8019,Famiglie per capacità di arrivare a fine mese,Households by ability to make ends meet


## 41_983/A.F.001001.KILLINJ

Il codice 41_983/A.F.001001.KILLINJ è una chiave SDMX composta da:

41_983: ID del dataflow

A.F.001001.KILLINJ: chiave della serie (tipicamente, concatenazione delle dimensioni)

In [2]:
import requests
import xml.etree.ElementTree as ET

url = "https://sdmx.istat.it/SDMXWS/rest/dataflow"
r = requests.get(url)
r.raise_for_status()
root = ET.fromstring(r.content)

ns = {
    "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
    "xml": "http://www.w3.org/XML/1998/namespace"
}

for df in root.findall(".//structure:Dataflow", ns):
    df_id = df.attrib['id']
    name_it = df.find(".//common:Name[@xml:lang='it']", ns)
    if "41_983" in df_id:
        print(f"Dataflow ID: {df_id}")
        print("Nome:", name_it.text if name_it is not None else "")


Dataflow ID: 41_983
Nome: Incidenti, morti e feriti - comuni


In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# === URL SDMX da cui scaricare ===
url = "https://sdmx.istat.it/SDMXWS/rest/data/41_983/A.F.001001.KILLINJ"

# === Namespaces XML ===
ns = {
    "generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic",
    "message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message"
}

# === Scarica e parse il file XML ===
r = requests.get(url)
root = ET.fromstring(r.content)

# === Estrai osservazioni ===
data = []
for obs in root.findall(".//generic:Series/generic:Obs", ns):
    year = obs.find("generic:ObsDimension", ns).attrib["value"]
    value = obs.find("generic:ObsValue", ns).attrib["value"]
    data.append((int(year), float(value)))

# === In DataFrame ===
df = pd.DataFrame(data, columns=["Anno", "Valore"])
print(df)
